In [40]:
import json
import pandas as pd

In [41]:
# 1. Load the GeoJSON file
file_path = 'Data/zipcodes.geojson'  # Replace with the actual file path
with open(file_path, 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

In [42]:
# 2. Extract ZIP codes from each feature
zip_codes = []
for feature in geojson_data['features']:
    properties = feature.get('properties', {})
    zipcode = properties.get('addr:postcode')
    if zipcode and zipcode.isdigit():  # Ensure ZIP is valid (e.g., numeric)
        zip_codes.append(zipcode)

In [43]:
# 3. Remove duplicates and sort the ZIP codes
unique_zip_codes = sorted(set(zip_codes))

In [44]:
# 4. Save ZIP codes to a CSV file
df = pd.DataFrame(unique_zip_codes, columns=['ZipCode'])
df.to_csv('zip_codes.csv', index=False)

print(f"Extracted {len(unique_zip_codes)} unique ZIP codes and saved to zip_codes.csv")

Extracted 10 unique ZIP codes and saved to zip_codes.csv


In [45]:
def combine_and_clean_csv(file1, file2, output_file=None):
    # 1️⃣ Load the two CSV files into DataFrames
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # 2️⃣ Combine both DataFrames
    combined_df = pd.concat([df1, df2], ignore_index=True)

    # 3️⃣ Remove duplicate rows (keep only unique rows based on 'zpid' column)
    cleaned_df = combined_df.drop_duplicates(subset=['zpid'])

    # 4️⃣ Save the cleaned DataFrame as a new CSV file (if output_file is provided)
    if output_file:
        cleaned_df.to_csv(output_file, index=False)
        print(f"✅ Combined CSV successfully saved as '{output_file}'")
    
    return cleaned_df

In [46]:
# Call the function with input files and specify the output file
cleaned_df_1 = combine_and_clean_csv('95340 (1).csv', '95340 (2).csv', '95340.csv')
cleaned_df_2 = combine_and_clean_csv('95341 (1).csv', '95341 (2).csv', '95341.csv')
cleaned_df_3 = combine_and_clean_csv('95348 (1).csv', '95348 (2).csv', '95348.csv')

✅ Combined CSV successfully saved as '95340.csv'
✅ Combined CSV successfully saved as '95341.csv'
✅ Combined CSV successfully saved as '95348.csv'


In [47]:
import pandas as pd
import os

def combine_csv_files(zip_codes, folder_path):
    """
    Combine multiple CSV files named after ZIP codes into one DataFrame,
    keeping only the 'Price' and 'Zipcode' columns.

    Parameters:
    zip_codes (list): List of ZIP codes corresponding to the CSV filenames.
    folder_path (str): Path to the folder containing the CSV files.

    Returns:
    pd.DataFrame: Combined DataFrame with 'Price' and 'Zipcode' columns only.
    """
    combined_df = pd.DataFrame()
    
    for zip_code in zip_codes:
        file_path = os.path.join(folder_path, f'{zip_code}.csv')
        try:
            df = pd.read_csv(file_path, usecols=['Price', 'Zipcode'])
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"✅ Successfully loaded and combined file: {file_path}")
        except Exception as e:
            print(f"❌ Failed to load file: {file_path}. Error: {e}")
    
    # Remove duplicates if any exist
    combined_df = combined_df.drop_duplicates()
    
    # Export the combined DataFrame as a single CSV file
    combined_df.to_csv('combined_sales_data.csv', index=False)
    print("✅ Combined CSV successfully saved as 'combined_sales_data.csv'")
    
    return combined_df

In [48]:

# List of ZIP codes with valid data
zip_codes = [95301, 95333, 95340, 95341, 95348, 95365, 95388]

# Specify the path to the folder containing the CSV files
folder_path = 'Data/UC Merced real estate'

# Call the function
combined_df = combine_csv_files(zip_codes, folder_path)

✅ Successfully loaded and combined file: C:/Users/PAVILION/Desktop/UC Merced real estate\95301.csv
✅ Successfully loaded and combined file: C:/Users/PAVILION/Desktop/UC Merced real estate\95333.csv
✅ Successfully loaded and combined file: C:/Users/PAVILION/Desktop/UC Merced real estate\95340.csv
✅ Successfully loaded and combined file: C:/Users/PAVILION/Desktop/UC Merced real estate\95341.csv
✅ Successfully loaded and combined file: C:/Users/PAVILION/Desktop/UC Merced real estate\95348.csv
✅ Successfully loaded and combined file: C:/Users/PAVILION/Desktop/UC Merced real estate\95365.csv
✅ Successfully loaded and combined file: C:/Users/PAVILION/Desktop/UC Merced real estate\95388.csv
✅ Combined CSV successfully saved as 'combined_sales_data.csv'


In [49]:
import pandas as pd

def calculate_average_prices(csv_file_path, export_path='average_prices_by_zip.csv'):
    """
    Calculate the average home price for each ZIP code after excluding outliers
    (bottom 10% and top 10% of prices).

    Parameters:
    csv_file_path (str): Path to the combined CSV file.
    export_path (str): Path to export the result as a CSV file.

    Returns:
    pd.DataFrame: DataFrame with 'Zipcode' and 'Average Price' columns.
    """
    # 1️⃣ Load the combined CSV file
    df = pd.read_csv(csv_file_path)
    
    # Ensure 'Price' is numeric (in case of any non-numeric values)
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    
    # Drop rows with missing prices
    df = df.dropna(subset=['Price'])

    # 2️⃣ Remove outliers (bottom 10% and top 10%) for each ZIP code
    average_prices = []
    for zip_code, group in df.groupby('Zipcode'):
        # Calculate quantiles (10th and 90th percentiles)
        lower_bound = group['Price'].quantile(0.1)
        upper_bound = group['Price'].quantile(0.9)
        
        # Filter prices within 10th to 90th percentile
        filtered_prices = group[(group['Price'] >= lower_bound) & (group['Price'] <= upper_bound)]
        
        if len(filtered_prices) > 0:
            # 3️⃣ Calculate the average price for this ZIP code
            avg_price = filtered_prices['Price'].mean()
            average_prices.append({'Zipcode': zip_code, 'Average Price': round(avg_price, 2)})
            print(f"✅ ZIP {zip_code}: Average price = {round(avg_price, 2)} (after removing outliers)")
        else:
            print(f"⚠️ No valid prices available for ZIP {zip_code} after outlier removal")
    
    #4️⃣ Create a DataFrame with the results
    result_df = pd.DataFrame(average_prices)
    
    #5️⃣ Export the results to a CSV file
    result_df.to_csv(export_path, index=False)
    print(f"✅ Results saved to '{export_path}'")
    
    return result_df


In [50]:
# Call the function with the path to the combined CSV file
combined_csv_path = 'combined_sales_data.csv'
average_prices_df = calculate_average_prices(combined_csv_path)


✅ ZIP 95301: Average price = 538761.46 (after removing outliers)
⚠️ No valid prices available for ZIP 95333 after outlier removal
✅ ZIP 95340: Average price = 399807.7 (after removing outliers)
✅ ZIP 95341: Average price = 390460.76 (after removing outliers)
✅ ZIP 95348: Average price = 441889.56 (after removing outliers)
✅ ZIP 95365: Average price = 340999.5 (after removing outliers)
✅ ZIP 95388: Average price = 718983.5 (after removing outliers)
✅ Results saved to 'average_prices_by_zip.csv'


In [51]:
import pandas as pd
import glob

def combine_rental_csvs(file_pattern='2bed_1bath_*.csv', output_file='combined_rental_data.csv'):
    """
    Combine multiple rental CSV files into one DataFrame, keeping only 'Price' and 'Zipcode' columns.
    
    Args:
    file_pattern (str): The pattern to match rental CSV files (default is '2bed_1bath_*.csv')
    output_file (str): Name of the combined output CSV file (default is 'combined_rental_data.csv')
    
    Returns:
    pd.DataFrame: A combined DataFrame with 'Price' and 'Zipcode' columns.
    """
    # 1️⃣ Get list of all CSV files matching the pattern
    csv_files = glob.glob(file_pattern)
    print(f"📁 Found {len(csv_files)} files to combine: {csv_files}")
    
    # 2️⃣ Combine all CSVs into one DataFrame
    combined_df = pd.DataFrame()
    for file in csv_files:
        try:
            df = pd.read_csv(file, usecols=['Price', 'Zipcode'])  # Only load 'Price' and 'Zipcode'
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"✅ Successfully loaded: {file}")
        except Exception as e:
            print(f"❌ Failed to load {file}: {e}")
    
    # 3️⃣ Remove duplicates
    combined_df = combined_df.drop_duplicates()
    
    # 4️⃣ Save the final combined DataFrame as a CSV file
    combined_df.to_csv(output_file, index=False)
    print(f"📁 Combined CSV successfully saved as '{output_file}'")
    
    return combined_df


In [52]:
# 🔥 Call the function to combine all 2bed_1bath CSVs
combined_df = combine_rental_csvs()

📁 Found 5 files to combine: ['2bed_1bath_95301.csv', '2bed_1bath_95340.csv', '2bed_1bath_95341.csv', '2bed_1bath_95348.csv', '2bed_1bath_95388.csv']
✅ Successfully loaded: 2bed_1bath_95301.csv
✅ Successfully loaded: 2bed_1bath_95340.csv
✅ Successfully loaded: 2bed_1bath_95341.csv
✅ Successfully loaded: 2bed_1bath_95348.csv
✅ Successfully loaded: 2bed_1bath_95388.csv
📁 Combined CSV successfully saved as 'combined_rental_data.csv'


In [53]:
import pandas as pd
import glob

def combine_and_analyze_rental_csvs(file_pattern='2bed_1bath_*.csv', output_file='combined_rental_data.csv'):
    """
    Combine rental CSVs, exclude outliers (bottom 10% and top 10%) of 'Price', 
    and calculate the average rent for each ZIP code.
    
    Args:
    file_pattern (str): The pattern to match rental CSV files (default is '2bed_1bath_*.csv')
    output_file (str): Name of the combined output CSV file (default is 'combined_rental_data.csv')
    
    Returns:
    pd.DataFrame: A DataFrame with average rent for each ZIP code.
    """
    # 1️⃣ Get list of all CSV files matching the pattern
    csv_files = glob.glob(file_pattern)
    print(f"📁 Found {len(csv_files)} files to combine: {csv_files}")
    
    # 2️⃣ Combine all CSVs into one DataFrame, keeping only 'Price' and 'Zipcode'
    combined_df = pd.DataFrame()
    for file in csv_files:
        try:
            df = pd.read_csv(file, usecols=['Price', 'Zipcode'])  # Only load 'Price' and 'Zipcode'
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"✅ Successfully loaded: {file}")
        except Exception as e:
            print(f"❌ Failed to load {file}: {e}")
    
    # 3️⃣ Remove duplicates
    combined_df = combined_df.drop_duplicates()
    
    # 4️⃣ Convert 'Price' to numeric (to ensure proper calculations)
    combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce')
    
    # 5️⃣ Remove NaN values in 'Price'
    combined_df = combined_df.dropna(subset=['Price'])

    # 6️⃣ Exclude outliers (bottom 10% and top 10%) for each ZIP code
    filtered_df = combined_df.groupby('Zipcode', group_keys=False).apply(
        lambda x: x[(x['Price'] > x['Price'].quantile(0.1)) & (x['Price'] < x['Price'].quantile(0.9))]
    ).reset_index(drop=True)  # 🔥 Reset index to avoid 'Zipcode' being an index
    
    # 7️⃣ Calculate the average rent for each ZIP code
    average_rent_per_zip = filtered_df.groupby('Zipcode', as_index=False)['Price'].mean()
    average_rent_per_zip.rename(columns={'Price': 'Average Rent'}, inplace=True)
    
    # 8️⃣ Save the combined and filtered DataFrame as a CSV file
    filtered_df.to_csv(output_file, index=False)
    print(f"📁 Filtered CSV successfully saved as '{output_file}'")
    
    # 9️⃣ Save the average rent per ZIP code as a separate CSV file
    average_rent_per_zip.to_csv('average_rent_per_zip.csv', index=False)
    print(f"📈 Average rent per ZIP successfully saved as 'average_rent_per_zip.csv'")
    
    return average_rent_per_zip, filtered_df

In [54]:
# 🔥 Call the function to combine, filter, and calculate average rent
average_rent, filtered_df = combine_and_analyze_rental_csvs()

📁 Found 5 files to combine: ['2bed_1bath_95301.csv', '2bed_1bath_95340.csv', '2bed_1bath_95341.csv', '2bed_1bath_95348.csv', '2bed_1bath_95388.csv']
✅ Successfully loaded: 2bed_1bath_95301.csv
✅ Successfully loaded: 2bed_1bath_95340.csv
✅ Successfully loaded: 2bed_1bath_95341.csv
✅ Successfully loaded: 2bed_1bath_95348.csv
✅ Successfully loaded: 2bed_1bath_95388.csv
📁 Filtered CSV successfully saved as 'combined_rental_data.csv'
📈 Average rent per ZIP successfully saved as 'average_rent_per_zip.csv'


In [55]:
def combine_and_clean_rental_data(path_pattern='1bed_1bath_*.csv'):
    """
    Combines rental CSV files for 1 bed, 1 bath units, removes outliers, 
    and calculates the average rent for each zip code.

    Parameters:
    path_pattern (str): The file path pattern to match rental CSV files.

    Returns:
    pd.DataFrame: DataFrame with the average rent for each zip code.
    """
    # 1️⃣ Load and Combine CSVs
    all_files = glob.glob(path_pattern)
    combined_df = pd.concat([pd.read_csv(file, usecols=['Price', 'Zipcode']) for file in all_files], ignore_index=True)

    # 2️⃣ Clean Data: Drop rows with missing or invalid prices
    combined_df = combined_df.dropna(subset=['Price', 'Zipcode'])
    combined_df = combined_df[combined_df['Price'].apply(lambda x: str(x).replace('.', '').isdigit())]
    combined_df['Price'] = combined_df['Price'].astype(float)

    # 3️⃣ Remove Outliers (bottom 10% and top 10%)
    def remove_outliers(df, col='Price'):
        lower_bound = df[col].quantile(0.1)
        upper_bound = df[col].quantile(0.9)
        return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    cleaned_df = combined_df.groupby('Zipcode', group_keys=False).apply(remove_outliers)

    # 4️⃣ Calculate the Average Rent for Each Zip Code
    avg_rent_per_zip = cleaned_df.groupby('Zipcode')['Price'].mean().reset_index()
    avg_rent_per_zip.rename(columns={'Price': 'Average Rent'}, inplace=True)

    # 5️⃣ Export Data to CSV (optional)
    avg_rent_per_zip.to_csv('average_rent_1bed_1bath.csv', index=False)

    print("✅ Average rent for each ZIP code saved in 'average_rent_1bed_1bath.csv'")
    return avg_rent_per_zip

In [56]:
# Run the function
avg_rent_df = combine_and_clean_rental_data()

✅ Average rent for each ZIP code saved in 'average_rent_1bed_1bath.csv'


In [57]:
avg_rent_df

,Zipcode,Average Rent
0,95333,925.000000
1,95340,1123.076923
2,95341,1100.000000
3,95348,1281.250000
4,95388,995.000000


In [58]:
import pandas as pd

def load_and_clean_data(file_path, price_column='Price', zip_column='Zipcode'):
    """
    Load and clean the CSV, keeping only the price and zip columns.
    
    Parameters:
    file_path (str): The path to the CSV file.
    price_column (str): The name of the price column (can be 'Price', 'Average Price', or 'Average Rent').
    zip_column (str): The name of the zip code column (default is 'Zipcode').
    
    Returns:
    pd.DataFrame: DataFrame with ZipCode and Average Price columns.
    """
    # Load the file, selecting only the necessary columns
    df = pd.read_csv(file_path, usecols=[price_column, zip_column])
    
    # Rename columns for consistent naming
    df.rename(columns={price_column: 'AvgPrice', zip_column: 'ZipCode'}, inplace=True)
    
    # Drop rows with missing values and ensure the data types are correct
    df = df.dropna(subset=['AvgPrice', 'ZipCode'])
    df['AvgPrice'] = pd.to_numeric(df['AvgPrice'], errors='coerce')
    df['ZipCode'] = pd.to_numeric(df['ZipCode'], errors='coerce')
    
    # Drop any rows that couldn't be converted to numbers
    df = df.dropna()
    
    return df

In [65]:
# 🏠 Load and Clean Home Price Data (3br/1ba)
home_prices = load_and_clean_data(file_path='average_prices_by_zip.csv', price_column='Average Price', zip_column='Zipcode')
home_prices.rename(columns={'AvgPrice': 'AvgHomePrice (3br/1ba)'}, inplace=True)

# 🏠 Load and Clean Rent Data (2br/1ba)
rent_2b1b = load_and_clean_data(file_path='average_rent_per_zip.csv', price_column='Average Rent', zip_column='Zipcode')
rent_2b1b.rename(columns={'AvgPrice': 'AvgRent (2br/1ba)'}, inplace=True)

# 🏠 Load and Clean Rent Data (1br/1ba)
rent_1b1b = load_and_clean_data(file_path='average_rent_1bed_1bath.csv', price_column='Average Rent', zip_column='Zipcode')
rent_1b1b.rename(columns={'AvgPrice': 'AvgRent (1br/1ba)'}, inplace=True)

# 📊 Merge All DataFrames
combined_df = home_prices.merge(rent_2b1b, on='ZipCode', how='outer')
combined_df = combined_df.merge(rent_1b1b, on='ZipCode', how='outer')

print("✅ Data is combined")
combined_df


✅ Data is combined


,ZipCode,AvgHomePrice (3br/1ba),AvgRent (2br/1ba),AvgRent (1br/1ba)
0,95301,538761.46,1341.666667,NaN
1,95333,737500.00,NaN,925.000000
2,95340,399807.70,1529.000000,1123.076923
3,95341,390460.76,1483.333333,1100.000000
4,95348,441889.56,1605.454545,1281.250000
5,95365,340999.50,NaN,NaN
6,95388,718983.50,1800.000000,995.000000


In [71]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# 🏠 Assume combined_df is already loaded and has the following columns:
# 'ZipCode', 'AvgHomePrice (3br/1ba)', 'AvgRent (2br/1ba)', 'AvgRent (1br/1ba)', 'Estimated Profit'

def calculate_mortgage(home_price, down_payment_rate=0.2, interest_rate=7.5, loan_term_years=30):
    """ Calculate the monthly mortgage payment. """
    loan_amount = home_price * (1 - down_payment_rate)
    monthly_interest_rate = (interest_rate / 100) / 12
    num_payments = loan_term_years * 12
    if monthly_interest_rate == 0:
        return loan_amount / num_payments
    mortgage_payment = loan_amount * (monthly_interest_rate * (1 + monthly_interest_rate) ** num_payments) / \
                       ((1 + monthly_interest_rate) ** num_payments - 1)
    return mortgage_payment

In [72]:


def calculate_estimated_profit(row):
    """ Calculate the profit for a given row. """
    home_price = row['AvgHomePrice (3br/1ba)']
    rent_2b1b = row['AvgRent (2br/1ba)']
    rent_1b1b = row['AvgRent (1br/1ba)']
    
    if pd.isna(home_price) or (pd.isna(rent_2b1b) and pd.isna(rent_1b1b)) or pd.isna(rent_1b1b):
        return None
    
    mortgage_payment = calculate_mortgage(home_price)
    monthly_rent_revenue = (rent_2b1b * 2) + rent_1b1b
    estimated_profit = monthly_rent_revenue - mortgage_payment
    return estimated_profit

In [73]:
def fill_missing_profits_with_regression(df):
    """ 
    Fill missing Estimated Profit using a regression model 
    and add a flag column 'IsProfitPredicted' to indicate if it was predicted.
    """
    # Step 1: Separate rows with and without Estimated Profit
    df_with_profit = df[df['Estimated Profit'].notna()]
    df_missing_profit = df[df['Estimated Profit'].isna()]
    
    if len(df_with_profit) > 1:  # Ensure we have enough data points to fit the model
        # Step 2: Impute missing values for 'AvgHomePrice (3br/1ba)'
        imputer = SimpleImputer(strategy='mean')  # Replace NaN with the mean of the column
        X_train = df_with_profit[['AvgHomePrice (3br/1ba)']]
        X_train = imputer.fit_transform(X_train)  # Impute missing values in training data
        
        y_train = df_with_profit['Estimated Profit']
        
        # Step 3: Train the regression model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Step 4: Handle missing profits in the data to be predicted
        X_missing = df_missing_profit[['AvgHomePrice (3br/1ba)']]
        X_missing = imputer.transform(X_missing)  # Impute missing values
        
        # Step 5: Predict Estimated Profit for missing rows
        predicted_profits = model.predict(X_missing)
        
        # Step 6: Update DataFrame with predicted profits
        df.loc[df['Estimated Profit'].isna(), 'Estimated Profit'] = predicted_profits
        
        # Step 7: Indicate that the Estimated Profit is predicted
        df.loc[df['Estimated Profit'].isna(), 'IsProfitPredicted'] = True
    
    return df

In [75]:
# 🧮 Calculate Estimated Profit for each row
combined_df['Estimated Profit'] = combined_df.apply(calculate_estimated_profit, axis=1)

# 🧮 Add column to mark profits that were directly calculated
combined_df['IsProfitPredicted'] = combined_df['Estimated Profit'].isna()  # Mark missing rows

# 🧮 Fill missing profits using the linear regression model
combined_df = fill_missing_profits_with_regression(combined_df)

# 📊 View the first few rows of the updated DataFrame
print("✅ Estimated Profit has been calculated and missing profits have been estimated.")


✅ Estimated Profit has been calculated and missing profits have been estimated.


In [78]:
combined_df

,ZipCode,AvgHomePrice (3br/1ba),AvgRent (2br/1ba),AvgRent (1br/1ba),Estimated Profit,IsProfitPredicted
0,95301,538761.46,1341.666667,NaN,1384.882231,True
1,95333,737500.00,NaN,925.000000,525.965829,True
2,95340,399807.70,1529.000000,1123.076923,1944.666167,False
3,95341,390460.76,1483.333333,1100.000000,1882.540039,False
4,95348,441889.56,1605.454545,1281.250000,2020.354359,False
5,95365,340999.50,NaN,NaN,2239.578010,True
6,95388,718983.50,1800.000000,995.000000,573.210443,False


In [77]:
# 📁 Export DataFrame to Excel
output_path = 'combined_real_estate_data.xlsx'  # You can change the file name or path if needed

# Export combined DataFrame to an Excel file
combined_df.to_excel(output_path, index=False)  # index=False prevents the index from being saved as a column

print(f"✅ DataFrame successfully exported to '{output_path}'")

✅ DataFrame successfully exported to 'combined_real_estate_data.xlsx'
